# <center>高级心理统计 (Advanced Statistics in Psych Sci) </center>  
## <center>《贝叶斯统计及其在Python中的实现》 (Bayesian inference with Python) </center>  
## <center>Instructor： 胡传鹏（博士）(Dr. Hu Chuan-Peng) </center>  
### <center>南京师范大学心理学院 (School of Psychology, Nanjing Normal University)  </center>

## <center> Part 1: Why Bayesian statistics  </center>

<center>  
	研究人类心理与行为的规律，容易吗？  
</center>

## Outlines  
* 1. 为什么要学习本课程 [Why Bayesia inference]  
* 2. 本课程的内容将是什么 [What is the syllabus]  
* 3. 如何学好这门课[How can I learn this course well]

##  1. 为什么要学习本课程 (Why Bayesia inference)  

### 1.1 为什么心理学需要更好的方法 (Why does psychological science need better methods?)  

#### 原因1: 人类认知与行为本身的复杂性

![Image Name](https://cdn.kesci.com/upload/image/rhdcyu860w.gif?imageView2/0/w/960/h/960)  


Source: https://www.science.org/toc/science/309/5731

**Q1: What is the Uiverse Made of (physics)**  

**Q2: What is the Biological Basis of Consciouness (psychological/cognitive science)**

##### Q:  
重要性和复杂度相似的问题，是否意味着研究方法也应该类似地复杂？

##### 物理学中的方法 (Methods in Physics):  

Example 1: Webb telescope (韦伯望远镜)  (**equipment**)


![Image Name](https://cdn.kesci.com/upload/image/rhdd0r46k3.png?imageView2/0/w/720/h/640)  


Example 2: Big-team science (CERN, the European Organization for Nuclear Research) ---- **equipment & practices**  

Example 3: **Mathematics**

##### 其他研究人类智能的领域所采用的方法 (Methods in other fields that also study "intelligence")  

**AI**  


![Image Name](https://cdn.kesci.com/upload/image/rhdd1sr5y2.png?imageView2/0/w/640/h/640)  


##### 心理科学的研究方法 [What do psychological scientists have?]  
你们能够想到的研究方法包括哪些？


![Image Name](https://cdn.kesci.com/upload/image/rhdd2dgwc8.png?imageView2/0/w/640/h/640)  


**实证研究：**  
* 质性研究  
* 观察法  
* 问卷  
* 行为实验  
* 眼动、生理数据记录  
* EEG/ERP/MEG  
* fMRI/PET/fNIRs  
* TMS/tDCS  
* ...

**统计方法：**  
* t-test  
* ANOVA  
* Correlation  
* Structural equation model (SEM)  
* ?

##### 相关方法课程：  
* 心理测量  
* 心理统计（包括SPSS等）  
* 实验心理学（包括E-prime等）  
* ？

* 更好的仪器  
* **更好的统计/数据分析**  
* 更好的实践 (e.g., big-team science)

#### 原因2: 更复杂的数据  

* 数据字化的时代，大数据  
* 神经成像/生理数据  
* 多模态的数据融合

### 1.2 确实有更好的统计方法

**贝叶斯统计 (Bayesian inference)  **  


![Image Name](https://cdn.kesci.com/upload/image/rhdf3bb12c.png?imageView2/0/w/640/h/640)  



* 灵活/强大/能用  
* 易用  
* 可拓展性强  
* 方便跨学科交流  
* ...

##### 灵活/强大/通用  

不需要解析解  

贝叶斯分析在多个学科中得到广泛应用，尤其是AI。

##### （相对）易用  

概率编程语言(Probabilistic Programming Languages)的发展和普及  


PPLs: *computational languages for statistical modeling*  

* PyMC  
* Stan  
* NumPyro  
* Pyro  
* BUGS  
* ...

大部分情况下，开发者使用它可以轻松地定义概率模型，然后程序会自动地求解模型。  


![Image Name](https://cdn.kesci.com/upload/image/rhdf4r9fbh.png?imageView2/0/w/640/h/640)  


Source: https://towardsdatascience.com/intro-to-probabilistic-programming-b47c4e926ec5  


##### 可拓展  

贝叶斯概念已经应用到以深度学习为中心的新技术的发展，包括深度学习框架(TensorFlow, Pytorch)，创建表示能力更强、数据驱动的模型

##### 方便交流  
大部分PPLs都有类似的数据结构，但是不同的学科使用的语言不同。  

心理学/社会科学/神经科学：  
* **PyMC3**  
* Stan  
* BUGS

<div style="text-align: center;">	 
	
## Part 2: Two examples  
	
</div> 

### 例1：社会关系地位与幸福感的关系  

实例的数据来自[ Many Labs 2 项目](osf.io/uazdm/)中的一个研究。  

该研究探究了社会关系地位对于幸福感的影响 “Sociometric status and well-being”， (Anderson, Kraus, Galinsky, & Keltner, 2012)。  

该数据集包括6905个被试的数据。

In [1]:
# import modules
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import xarray as xr

%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")
rng = np.random.default_rng(1234)

import matplotlib
matplotlib.rcParams['figure.figsize'] = [4, 3]

In [2]:
# 导入数据, 注意在和鲸和在自己电脑（本地）时有差异）
try:
  SMS_data = pd.read_csv('/home/mw/input/Bayesian3285/data_chp1_SMS_Well_being.csv')
except:
  SMS_data = pd.read_csv('data/data_chp1_SMS_Well_being.csv')
SMS_data = SMS_data[['uID','variable','factor','Country']]
# 把数据分为高低两种社会关系的地位的子数据以便画图与后续分析
plot_data = [
    sorted(SMS_data.query('factor=="Low"').variable[0:3000]),
    sorted(SMS_data.query('factor=="High"').variable[0:3000])]

#### 通过画图对于两种社会关系地位对幸福感的影响  

图中横坐标代表高低两种社会关系地位，纵坐标代表了主观幸福感评分。

In [3]:
# import matplotlib
# a = sorted([f.name for f in matplotlib.font_manager.fontManager.ttflist])

# for i in a:
#    print(i)

# 字体样式
font = {'family' : 'Source Han Sans CN'}
# 具体使用
plt.rc('font',**font)

In [4]:
# 画图对比两种社会地位对幸福感的影响
def adjacent_values(vals, q1, q3):
    upper_adjacent_value = q3 + (q3 - q1) * 1.5
    upper_adjacent_value = np.clip(upper_adjacent_value, q3, vals[-1])

    lower_adjacent_value = q1 - (q3 - q1) * 1.5
    lower_adjacent_value = np.clip(lower_adjacent_value, vals[0], q1)
    return lower_adjacent_value, upper_adjacent_value

def set_axis_style(ax, labels):
    ax.xaxis.set_tick_params(direction='out')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xticks(np.arange(1, len(labels) + 1), labels=labels)
    ax.set_xlim(0.25, len(labels) + 0.75)
    ax.set_xlabel('社会关系地位')

fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(9, 4), sharey=True)

parts = ax1.violinplot(
        plot_data, showmeans=False, showmedians=False,
        showextrema=False)

for pc in parts['bodies']:
    pc.set_facecolor('#D43F3A')
    pc.set_edgecolor('black')
    pc.set_alpha(1)

quartile1, medians, quartile3 = np.percentile(plot_data, [25, 50, 75], axis=1)
whiskers = np.array([
    adjacent_values(sorted_array, q1, q3)
    for sorted_array, q1, q3 in zip(plot_data, quartile1, quartile3)])
whiskers_min, whiskers_max = whiskers[:, 0], whiskers[:, 1]

inds = np.arange(1, len(medians) + 1)
ax1.scatter(inds, medians, marker='o', color='white', s=30, zorder=3)
ax1.vlines(inds, quartile1, quartile3, color='k', linestyle='-', lw=5)
ax1.vlines(inds, whiskers_min, whiskers_max, color='k', linestyle='-', lw=1)

# set style for the axes
labels = ['低','高']
plt.xticks(np.arange(2)+1, labels)
plt.xlabel('社会关系地位')
plt.ylabel('幸福感')

plt.show()

<Figure size 900x400 with 1 Axes>

#### 通过*t*检验，分析两种社会关系地位下幸福感的差异  

结果发现，两种社会关系水平下被试的主观幸福感边缘显著，*t*(6903) = -1.76, *p* = .08。

In [5]:
from scipy import stats
SMS_low = SMS_data.query('factor=="Low"').variable.values
SMS_high = SMS_data.query('factor=="High"').variable.values
print(
    f"低社会关系：{np.around(np.mean(SMS_low),3)} ± {np.around(np.std(SMS_low),2)}；",
    f"高社会关系：{np.around(np.mean(SMS_high),3)} ± {np.around(np.std(SMS_high),2)}")
    
stats.ttest_ind(
    a= SMS_low,
    b= SMS_high, 
    equal_var=True)

低社会关系：0.014 ± 0.66； 高社会关系：-0.014 ± 0.67


TtestResult(statistic=1.7593310889762195, pvalue=0.07856558333862036, df=6903.0)

#### 通过贝叶斯推断替代*t*检验  

零假设显著性检验（Null hypothesis significance test, NHST）的框架之下，*t*检验只提供了一个二分的结果：拒绝或者无法拒绝$H_0$。 但 *p* = 0.078这样的结果无法支持$H_0$  

贝叶斯推断是否可以带来不一样的结果？  

一个简单的线性模型：  

1. 通过建立线性模型去替代原本的*t*检验模型。  

2. 通过PyMC对后验进行采样  

3. 通过Arviz对结果进行展示，辅助统计推断

In [6]:
# 通过pymc建立基于贝叶斯的线性模型
x = pd.factorize(SMS_data.factor)[0] # high为0，low为1

with pm.Model() as linear_regression:
    sigma = pm.HalfCauchy("sigma", beta=2)
    beta0 = pm.Normal("beta0", 0, sigma=5)
    beta1 = pm.Normal("beta1", 0, sigma=5)
    x = pm.MutableData("x", x)
    # μ = pm.Deterministic("μ", β0 + β1 * x)
    pm.Normal("y", mu=beta0 + beta1 * x, sigma=sigma, observed=SMS_data.variable)

可以通过pymc自带的可视化工具将模型关系可视化。  

x 为自变量，其中1为低社会关系，0为高社会关系。  

参数 $\beta0$ 是线性模型的截距，而 $\beta1$ 是斜率。  

截距代表了高社会关系地位被试的幸福感；而截距加上斜率表示低社会关系地位被试的幸福感。  

参数$sigma$是残差，因变量$y$即主观幸福感。  

模型图展示了各参数通过怎样的关系影响到因变量。

In [7]:
pm.model_to_graphviz(linear_regression)

In [8]:
# 模型拟合过程 (mcmc采样过程)
with linear_regression:
    idata = pm.sample(2000, tune=1000, target_accept=0.9, return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, beta0, beta1]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 23 seconds.


#### 参数的后验分布  
这里的模型分析结果展示了各参数的分布(后验)情况

In [9]:
az.plot_trace(idata);

<Figure size 1200x600 with 6 Axes>

下图反应了参数β1的可信度，即两个社会关系下幸福感差异的可信度。  

结果显示，两个社会关系下幸福感差异的可信度为96%。

In [10]:
(idata.posterior.beta1 > 0).mean().values

array(0.96075)

In [11]:
axes = az.plot_posterior(idata, var_names=['beta1'], kind='hist',ref_val=0)

<Figure size 400x300 with 1 Axes>

In [12]:
axes = az.plot_posterior(idata, var_names=['beta1'], kind='hist', rope = [-0.1, 0.1], hdi_prob=.95)

<Figure size 400x300 with 1 Axes>

#### 模型诊断  

通过模型思维进行数据分析需要注意模型检验，即检验模型是否能有效的反应数据的特征。

下表格为模型参数的基本信息：  

mean和sd 为各参数的均值和标准差；  
hdi 3%-97% 为参数分布的可信区间；  
msce mean和sd 为mcmc采样标准误统计量的均值和标准差；  
ess bulk和tail 反应了mcmc采样有效样本数量相关性能；  
r hat 为参数收敛性的指标。

In [13]:
az.summary(idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta0,-0.014,0.011,-0.035,0.008,0.0,0.0,4850.0,4966.0,1.0
beta1,0.028,0.016,-0.002,0.057,0.0,0.0,5038.0,5125.0,1.0
sigma,0.661,0.006,0.650,0.671,0.0,0.0,5129.0,5513.0,1.0


### 后验预测检验 ppc (posterior predictive check)

In [14]:
x,x_levels = SMS_data.factor.factorize()

with linear_regression:
    pm.set_data({"x": x})
    idata_ppc = pm.sample_posterior_predictive(
        idata, 
        var_names=["y"],
        return_inferencedata=True,
        predictions=True,
        extend_inferencedata=True
)

Sampling: [y]


In [15]:
idata_ppc

Inference data with groups:
	> posterior
	> predictions
	> sample_stats
	> observed_data
	> constant_data
	> predictions_constant_data

In [16]:
df_ppc = pd.DataFrame({
    "x": idata_ppc.constant_data.x.values, 
    "y_mean": idata_ppc.predictions.y.mean(["chain", "draw"]).values # 合并 chains 和 draw 为均值
})
df_ppc

,x,y_mean
0,0,-0.015016
1,0,-0.016379
2,0,-0.006570
3,0,-0.028285
4,0,-0.018867
...,...,...
6900,1,0.006583
6901,1,0.027325
6902,1,0.006328
6903,1,0.017191


In [17]:
df_ppc[df_ppc.x==1].y_mean.values

array([0.01355639, 0.01519054, 0.01070339, ..., 0.00632784, 0.01719058,
       0.00254927])

In [18]:
labels = ['低', '高']
obs_low = SMS_data.query('factor=="Low"').variable
obs_high = SMS_data.query('factor=="High"').variable
ppc_low = df_ppc[df_ppc.x==1].y_mean.values
ppc_high = df_ppc[df_ppc.x==0].y_mean.values

fig, ax = plt.subplots()
part1 = ax.violinplot(
    [list(obs_low),list(obs_high)], 
    [1,4], points=100, widths=0.3, 
    showmeans=True, showextrema=True, showmedians=True)
part2 = ax.violinplot(
    [list(ppc_low),list(ppc_high)], 
    [2,5], points=100, widths=0.3, 
    showmeans=True, showextrema=True, showmedians=True)
part1['bodies'][0].set_label('观测数据')
part2['bodies'][0].set_label('预测数据')
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('幸福感')
ax.set_title('Posterior predictive check')
plt.xticks([1.5,4.5], labels)
ax.legend()

fig.tight_layout()
plt.show()

/tmp/ipykernel_81/3477623782.py:24: UserWarning: The figure layout has changed to tight
  fig.tight_layout()


<Figure size 400x300 with 1 Axes>

### 例2: 贝叶斯推断在认知模型中的应用

在实验数据的收集时，研究者往往会采集个体反应的正确率与反应时。  

而传统分析方法并不能同时对两种数据进行分析，从而推断潜在的认知机制。比如，个体是否愿意牺牲更多的反应时间去获得一个更准确的判断。  

认知模型能有效的弥补这一问题，比如 drift-diffusion model, DDM。


![DDM1](https://cdn.kesci.com/upload/image/rhb2957an5.png?imageView2/0/w/960/h/960)  


<div style="text-align: center;">	 
	
## Part 3: About this course  
	
</div> 

# 2. 课程内容  
### 2.0 课时  
9.05 ～ 12.30，18周，十一假期可能会被放假一次  

### 2.1 教学目标：  

（1）使用Python进行基本的数据处理  

（2）理解贝叶斯推断的基本原理；  

（2）了解PyMC3的语法和结构，并能应用于相对简单的情境  


### 2.2 考核方式：  

#### 考勤  
10%  




#### 小作业 (45%)  
	
（1）使用numpy与pandas进行基本的数据预处理；  

（2）使用pymc对随机变量进行简单线性回归模型的建模；  

（3）使用arviz进行统计推断。

#### 大作业 (45%)   
真实的数据分析  
* 合作完成  
* 包括代码与文字报告  
* 进行汇报  
* 标准：分工合理、数据分析流程完整、汇报展示清晰美观


### 2.3 课程风格：  
	（1）内容有挑战、考核不复杂  
	（2）1/3一节课展示或互动抄代码  
	（3）专门设有答疑时间，助教给大家答疑解惑  





### 2.4 课程大纲  
### Intro  
1: 课程介绍（为什么要用贝叶斯/PyMC3，展示一个回归分析例子，课程安排）  

### I Basics：  
2: 贝叶斯公式  
* 单一事件的贝叶斯模型(先验、似然、分母和后验)  
* 随机变量的贝叶斯模型(先验、似然、分母和后验)  

3: 建立一个完成的贝叶斯模型：Beta-Binomial model  
* Beta先验  
* Binomial与似然  
* Beta-Binomial model  

4: 贝叶斯模型的特点：数据-先验与动态更新  
* 先验与数据对后验的影响  
* 数据顺序不影响后验  

5: 经典的贝叶斯模型：Conjugate families  
* Gamma-Poisson  
* Normal-Normal  

### II 近似后验估计  
6: 近似后验的方法  
* 网格法  
* MCMC  

7: 深入一种MCMC算法  
* M-H 算法  

8: 基于后验的推断  
* 后验估计  
* 假设检验  
* 后验预没  

### III Bayesian回归模型  
9: 简单的Bayesian线性回归模型  
* 建立模型  
* 调整先验  
* 近似后验  
* 基于近似后验的推断  
* 序列分析  

10: 对回归模型的评估  
* 评估模型的标准  
* 对简单线性模型的估计  
  
11: 扩散线性模型  
* 多自变量的线性回归  

12: GLM: Possion & Negative Binomial Regression  

13: GLM: Logistic Regression  

14: GLM: Naive Bayes Classification  

15: Hierachical Bayesian Model

### 2.5 参考书  

<div style="display: flex;">  
	<img src="https://cdn.kesci.com/upload/s0i1y2dj22.jpeg?imageView2/0/w/320/h/320" alt="BayesRule" style="flex: 1;">  
	<img src="https://cdn.kesci.com/upload/s0i1xtsmcj.jpg?imageView2/0/w/320/h/320" alt="BayesianInPython" style="flex: 1;">  
  <img src="https://cdn.kesci.com/upload/s0i1xcemfe.png?imageView2/0/w/320/h/320" alt="StatsRethinking" style="flex: 1;">  
	<img src="https://cdn.kesci.com/upload/s0i26ml0bk.jpg?imageView2/0/w/320/h/320" alt="StudentGuide" style="flex: 1;">  
</div>